# WhisperX48_local: 用于本地运行WhisperX48的Jupyter Notebook脚本


In [ ]:
'''
This script is referenced and modified minorly from Ayanaminn's N46Whisper project under MIT license. 

MIT License

Copyright (c) 2022 Ayanaminn

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.
'''

import os
import ffmpeg
import subprocess
import torch
import whisperx
import time
from pathlib import Path
import sys


model_size = "small"  # @param ["base","small","medium", "large"]
language = "ja"  # @param {type:"string"}
#sub_style = "default"  # @param ["default", "ikedaCN", "kaedeCN","sugawaraCN"]
#compression_ratio_threshold = 2.4 # @param {type:"number"}
#no_speech_threshold = 0.6 # @param {type:"number"}
#logprob_threshold = -1.0 # @param {type:"number"}
#condition_on_previous_text = "True" # @param ["True", "False"]

output_dir = "./files/"  # 默认的音频文件输入和字幕文件输出路径 Path for input audio file and output subtitle by default
file_name = "sample1.wav"  # 在这里输入音频文件名 Name of the audio file
audio_file = output_dir + file_name

device = "cuda"
torch.cuda.empty_cache()
print('加载Whisper模型 Loading Whisper model...')
model = whisperx.load_model(model_size, device)

#Original whisper transcribe
tic = time.time()
print('识别中 Transcribe in progress...')
result = model.transcribe(audio_file, language =language)

#Load alignment model and metadata
print('加载调整模型 Load alignment model...')
#model_id = "jonatasgrosman/wav2vec2-large-xlsr-53-japanese"
model_a, metadata = whisperx.load_align_model(language_code="ja", device=device)

#Align whisper output
print('调整识别结果 Align whisper output...')
result_aligned = whisperx.align(result["segments"], model_a, metadata, audio_file, device)

toc = time.time()
print('识别完毕 Done')
print(f'Time consumpution {toc-tic}s')

#Write SRT file
from whisperx.utils import write_srt
with open(Path(output_dir) / (audio_file + ".srt"), "w", encoding="utf-8") as srt:
    write_srt(result["segments"], file=srt)
print('字幕生成完毕 Subtitle generated!')

torch.cuda.empty_cache()

<font size="2">  

Last modified 2023-03-08  
  
Author: ifeimi ♢ Email me: yfwu0202 AT gmail.com

